In [43]:
import pandas as pd

In [44]:
import openpyxl

In [45]:
food_df = pd.read_excel('국가표준식품성분.xlsx')
code_df = pd.read_excel('식품코드,국문명,영문명.xlsx')
unit_df = pd.read_excel('영양성분표기및단위.xlsx')
food_dropnull_df = pd.read_excel('식품성분결측값제거.xlsx')

pd.set_option('display.max_columns',None)

In [46]:
pd.set_option('display.max_rows',None)

search_food = food_df
# code = 'C10200'

#근사치 값 제거
search_food = search_food[~search_food['ref'].str.contains('japan', case=False)]
# drop_df = search_food[search_food['ref'].str.contains('JAPAN')].index
# search_food = search_food.drop(drop_df)

# 말린 것 제거
search_food = search_food[~search_food['name'].str.contains('말린것', case=False)]
# drop_df = search_food[search_food['name'].str.contains('말린것')].index
# search_food = search_food.drop(drop_df)

# 가루 제거
search_food = search_food[~search_food['name'].str.contains('가루', case=False)]
# drop_df = search_food[search_food['name'].str.contains('가루')].index
# search_food = search_food.drop(drop_df)

search_food = search_food.fillna(0)
# search_food = search_food.sort_values(by = code , ascending = False)
# search_food[['name',code]].head(200)


In [47]:
search_food['group'].unique()

array(['곡류 및 그 제품', '감자류 및 전분류', '당류', '두류', '견과류 및 종실류', '채소류', '버섯류',
       '과일류', '육류 및 그 제품', '난류', '어패류 및 그 제품', '해조류', '우유 및 그 제품', '유지류',
       '차류', '음료류', '주류', '조미료류', '조리가공식품류', '기타'], dtype=object)

In [48]:
unique_names = search_food['group'].unique()

# 고유한 이름에 인덱스를 사용하여 새로운 이름을 지정합니다.
name_to_group = {name: f"group{index}" for index, name in enumerate(unique_names)}
print(name_to_group)
# map 함수를 사용하여 'Name' 열의 값을 새로운 그룹 이름으로 변경합니다.
search_food['group'] = search_food['group'].map(name_to_group)
print(search_food)


{'곡류 및 그 제품': 'group0', '감자류 및 전분류': 'group1', '당류': 'group2', '두류': 'group3', '견과류 및 종실류': 'group4', '채소류': 'group5', '버섯류': 'group6', '과일류': 'group7', '육류 및 그 제품': 'group8', '난류': 'group9', '어패류 및 그 제품': 'group10', '해조류': 'group11', '우유 및 그 제품': 'group12', '유지류': 'group13', '차류': 'group14', '음료류': 'group15', '주류': 'group16', '조미료류': 'group17', '조리가공식품류': 'group18', '기타': 'group19'}
      DB색인    group                              name        ref  A10100  \
0        1   group0                   귀리, 겉귀리, 도정, 생것   농진청('16)     380   
1        2   group0                   귀리, 쌀귀리, 도정, 생것   농진청('20)     388   
2        3   group0               귀리, 쌀귀리, 대양, 도정, 생것   농진청('20)     386   
3        4   group0               귀리, 쌀귀리, 조양, 도정, 생것   농진청('20)     390   
4        5   group0                    귀리, 쌀귀리, 도정, 밥   농진청('20)     218   
5        6   group0                귀리, 쌀귀리, 대양, 도정, 밥   농진청('20)     215   
6        7   group0                귀리, 쌀귀리, 조양, 도정, 밥   농진청('20)     221   
7    

In [49]:
for i in range(20):
    print('group', i, unique_names[i])

group 0 곡류 및 그 제품
group 1 감자류 및 전분류
group 2 당류
group 3 두류
group 4 견과류 및 종실류
group 5 채소류
group 6 버섯류
group 7 과일류
group 8 육류 및 그 제품
group 9 난류
group 10 어패류 및 그 제품
group 11 해조류
group 12 우유 및 그 제품
group 13 유지류
group 14 차류
group 15 음료류
group 16 주류
group 17 조미료류
group 18 조리가공식품류
group 19 기타


In [50]:
search_food = search_food.rename(columns={'DB색인': 'DB_index'})

In [51]:
code_df = code_df.rename(columns={'DB색인': 'DB_index', '식품코드' : 'food_code','식품명_국문명' : 'food_name_kr', '식품명_영문명' : 'food_name_en', '학명' : 'scientific_name'})

In [52]:
unit_df = unit_df.rename(columns={'code' : 'nut_code', '성분_국문명' : 'nut_name_kr', '성분_영문명' : 'nut_name_en', '성분_약어' : 'nut_abbreviation', '단위' : 'unit'})

In [53]:
code_df.columns, unit_df.columns

(Index(['DB_index', 'food_code', 'food_name_kr', 'food_name_en',
        'scientific_name'],
       dtype='object'),
 Index(['nut_code', 'nut_name_kr', 'nut_name_en', 'nut_abbreviation', 'unit',
        'growth_reproduction_minimum', 'adult_maintenance_Minimum', 'maximum'],
       dtype='object'))

In [54]:
filter_numbers = [2299, 2311, 2316, 2329, 2382, 2387, 2533, 2547, 2551, 2598, 2599, 2620, 2630]


In [55]:
for i in range(13):
    search_food = search_food[search_food['DB_index'] != filter_numbers[i]]

In [56]:
import sqlite3

In [58]:
db_file = 'db2.sqlite3'
connection = sqlite3.connect(db_file)

In [59]:
table_name = 'food_national'
search_food.to_sql(table_name, connection, if_exists='replace', index=False)


2591

In [60]:
table_name = 'food_name'
code_df.to_sql(table_name, connection, if_exists='replace', index=False)


3270

In [61]:
table_name = 'food_nut'
unit_df.to_sql(table_name, connection, if_exists='replace', index=False)


134

In [19]:
unit_df.columns

Index(['nut_code', 'nut_name_kr', 'nut_name_en', 'nut_abbreviation', 'unit',
       'growth_reproduction_minimum', 'adult_maintenance_Minimum', 'maximum'],
      dtype='object')

In [20]:
unit_df.fillna(0, inplace=True)

In [21]:
max_nutrient = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4500.0, 0.0, 0.0, 4000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 500.0, 0.0, 2750.0, 18750.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 30.0, 2.0]
growth_nutrient = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3000.0, 22.0, 150.0, 2500.0, 1500.0, 800.0, 25.0, 3.1, 1.8, 90.0, 0.0, 250.0, 375.0, 0.0, 0.0, 0.56, 1.3, 3.4, 0.0, 0.0, 0.0, 3.0, 0.0, 0.38, 0.0, 0.0, 54.0, 0.0, 7.0, 0.0, 3.125, 0.0, 0.0, 8.39, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1780.0, 3230.0, 2250.0, 880.0, 2080.0, 2600.0, 500.0, 1700.0, 1100.0, 2500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3300.0, 200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
adult_nutrient = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1250.0, 10.0, 150.0, 1000.0, 1500.0, 200.0, 20.0, 1.83, 1.25, 80.0, 0.0, 250.0, 375.0, 0.0, 0.0, 0.56, 1.3, 3.4, 0.0, 0.0, 0.0, 3.0, 0.0, 0.38, 0.0, 0.0, 54.0, 0.0, 7.0, 0.0, 3.125, 0.0, 0.0, 8.39, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 950.0, 1700.0, 1580.0, 830.0, 1130.0, 1200.0, 400.0, 1230.0, 480.0, 1280.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2800.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [22]:
import sqlite3

# SQLite 데이터베이스에 연결합니다.
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# DB_index가 10인 행을 검색하는 쿼리를 작성합니다.
query = "SELECT * FROM food_national WHERE DB_index = 10"

# 쿼리를 실행하고 결과를 가져옵니다.
cursor.execute(query)
result = cursor.fetchone()
result_list = list(result)
result_list = result[4:]

# 결과를 출력합니다.
print(result_list)

# 리소스를 정리합니다.
cursor.close()
conn.close()


(371, 11.3, 13.63, 3.77, 1.63, 69.65, 0.46, 0.46, 0.0, 0.0, 0.0, 0.0, 0.0, 4.7, 0.4, 4.2, 21, 4.78, 155, 370, 301, 5, 3.2, 0.68, 0.972, 3.41, 35.46, 0.36, 0, '0', '5', 0.634, 0.068, 1.162, 4.321, 0.831, 0.331, 0.293, 0.314, 0.145, 1.84, 50, 50, 0.0, 0.0, '0', 0.0, 0.0, 0.0, 0.29, 0.1, 0.08, 1.4, 0.12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11176, 4697, 488.0, 1456.0, 166.0, 243, 647.0, 314.0, 190, 615.0, 245, 333.0, 222.0, 110, 1228.0, 672.0, 2535.0, 252.0, 786.0, 674.0, 0.0, 0.0, 3.39, 2.23, 0.51, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 417.59, 1.14, 47.02, 18.32, 1.42, 16.02, 0.0, 12.91, 2.88, 0.64, 0.0, 5.97, 0.0, 577.36, 38.44, 16.98, 0.0, 1.28, 2.24, 2187.57, 43.03, 0.0, 1.91, 0.0, 0.0, 3.3, 0.0, 0.0, 0.0, 0.0, 0.04, 2.19, 0.0, 0.0, 3.35, 0.88, 0.0, 0.0)


# json Load

In [23]:
import json

json_data = '[{"name": "돼지고기, 등심, 생것", "unit": "0"}, {"name": "달걀, 생것", "unit": "0"}, {"name": "멥쌀, 백미, 밥", "unit": "0"}, {"name": "고구마, 생것", "unit": "0"}, {"name": "호박, 단호박, 생것", "unit": "0"}]'
data_list = json.loads(json_data)

for data in data_list:
    print(data["name"], data["unit"])


돼지고기, 등심, 생것 0
달걀, 생것 0
멥쌀, 백미, 밥 0
고구마, 생것 0
호박, 단호박, 생것 0


# 음식 항목 불러오기

In [24]:
import sqlite3

# SQLite 데이터베이스에 연결합니다.
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

input_list = list()

for data in data_list:
    searching_food_db_template = "SELECT * FROM food_national WHERE name = '{}'"
    user_input = data["name"]

    searching_food_db = searching_food_db_template.format(user_input)


    # 쿼리를 실행하고 결과를 가져옵니다.
    cursor.execute(searching_food_db)
    result = cursor.fetchone()
    temp = list(result)
    del temp[0:2]
    temp.pop(1)
    input_list.append(temp)


# 리소스를 정리합니다.
cursor.close()
conn.close()
# 결과를 출력합니다.
print(len(input_list[0]))




133


nut_data

In [25]:
max_nutrient = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4500.0, 0.0, 0.0, 4000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 500.0, 0.0, 2750.0, 18750.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 30.0, 2.0]
growth_nutrient = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3000.0, 22.0, 150.0, 2500.0, 1500.0, 800.0, 25.0, 3.1, 1.8, 90.0, 0.0, 250.0, 375.0, 0.0, 0.0, 0.56, 1.3, 3.4, 0.0, 0.0, 0.0, 3.0, 0.0, 0.38, 0.0, 0.0, 54.0, 0.0, 7.0, 0.0, 3.125, 0.0, 0.0, 8.39, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1780.0, 3230.0, 2250.0, 880.0, 2080.0, 2600.0, 500.0, 1700.0, 1100.0, 2500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3300.0, 200.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
adult_nutrient = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1250.0, 10.0, 150.0, 1000.0, 1500.0, 200.0, 20.0, 1.83, 1.25, 80.0, 0.0, 250.0, 375.0, 0.0, 0.0, 0.56, 1.3, 3.4, 0.0, 0.0, 0.0, 3.0, 0.0, 0.38, 0.0, 0.0, 54.0, 0.0, 7.0, 0.0, 3.125, 0.0, 0.0, 8.39, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 950.0, 1700.0, 1580.0, 830.0, 1130.0, 1200.0, 400.0, 1230.0, 480.0, 1280.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2800.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [26]:
def caloric_calculator(kg, cal_weight):
  MER = 132 * (kg ** 0.75) # 대사체중 
  # MER = 132 * kg ^ 0.75 # kcal x kg ^ 0.75
  #DER = MER * ALL
  calories = MER * cal_weight
  return round(calories)

In [27]:
cal_weight = 1

#input
kg = 15
Neutered = True
Obese = True
activate = 2
Anxious = True
Senior = True
if Neutered:
    cal_weight = cal_weight *0.75
if Obese:
    cal_weight = cal_weight * 0.45
if activate == 0:
    cal_weight = cal_weight * 0.72
if activate == 1:
    cal_weight = cal_weight * 1.06
if activate == 2:
    cal_weight = cal_weight * 1.36
if Anxious:
    cal_weight = cal_weight * 1.2
if Senior:
    cal_weight = cal_weight * 0.8

In [28]:
min_caloric = caloric_calculator(kg, cal_weight)
max_caloric = min_caloric + 50
protein_min_cal, fat_min_cal, carbohydrate_min_cal = min_caloric * 0.4 / 4, min_caloric * 0.2 / 9, min_caloric * 0.4 / 4
protein_max_cal, fat_max_cal, carbohydrate_max_cal = max_caloric * 0.4, max_caloric * 0.2, max_caloric * 0.4

nutrient_list = [['A10300', '단백질', 'g', protein_min_cal, 0], ['A10400', '지방 ', 'g', fat_min_cal, 0], ['A10600', '탄수화물', 'g', carbohydrate_min_cal, 0], ['A10100', '에너지', 'kcal', min_caloric, max_caloric]]
nutrient_list

[['A10300', '단백질', 'g', 44.300000000000004, 0],
 ['A10400', '지방 ', 'g', 9.844444444444445, 0],
 ['A10600', '탄수화물', 'g', 44.300000000000004, 0],
 ['A10100', '에너지', 'kcal', 443, 493]]

In [29]:
"""The Stigler diet problem.

A description of the problem can be found here:
https://en.wikipedia.org/wiki/Stigler_diet.
"""
from ortools.linear_solver import pywraplp



"""Entry point of the program."""
# Instantiate the data problem.
# Nutrient minimums.
##코드, 이름, 단위, 최소, 최대
nutrients = nutrient_list

# 식재료 이름, 영양소 코드 
data = input_list

# Instantiate a Glop solver and naming it.
solver = pywraplp.Solver.CreateSolver('GLOP')
if not solver:
  print('solver 오류')

# 변수의 값의 최소값, 변수의 값의 최대값, 변수의 이름 // solver.infinity 는 무한대를 나타낸다. 즉 최대값의 한계가 정해져 있지 않다.
foods = []
print('----------- 고정 재료 목록 -----------')
for item in input_list:
    if item[1] != 0:
      food = solver.NumVar(item[1]/100, solver.infinity() , item[0])
      print(item)
    else:
      food = solver.NumVar(0.0, solver.infinity(), item[0])
    foods.append(food)
print('-----------------------------------------')
print('Number of variables =', solver.NumVariables())

# Create the constraints, one per nutrient.
#Constraint(0,2,'ct')는 목적함수가 가질 수 있는 값의 범위를 지정하는 것이다. 기본적으로 변수들의 linear 결합을 가정하고 있다.
#SetCoefficient로 지정해주면 그 선형 결합을 제약 조건에 사용하는 것이다.
constraints = []
for i, nutrient in enumerate(nutrients):
  ##제약조건 생성
    #constraints.append(solver.Constraint(nutrient[3], solver.infinity()))
    if nutrient[4] != 0 : 
      constraints.append(solver.Constraint(nutrient[3], nutrient[4]))
    else :
      constraints.append(solver.Constraint(nutrient[3], solver.infinity()))
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i + 1])

print('Number of constraints =', solver.NumConstraints())

# Objective function: Minimize the sum of (gram-normalized) foods
objective = solver.Objective()
for food in foods:
    objective.SetCoefficient(food, 100)
objective.SetMinimization()

status = solver.Solve()

# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)

calculated_recipe = []
# Display the amounts (in 100 grams) to purchase of each food.
nutrients_result = [0] * len(nutrients)
print('\n식재료 무게:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        calculated_recipe.append((data[i][0],round(100 * food.solution_value())))
        print('{}: {}'.format(data[i][0], 100. * food.solution_value()))
        for j, _ in enumerate(nutrients):
            nutrients_result[j] += data[i][j + 1] * food.solution_value()
print('\n무게 총합: {:.4f}g'.format(objective.Value()))

print('\nNutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('[{}] {} : {:.2f}{} (min {} , max {})'.format(nutrient[0], nutrient[1], nutrients_result[i],
                                        nutrient[2],nutrient[3],nutrient[4]))

print('\nAdvanced usage:')
print('Problem solved in ', solver.wall_time(), ' milliseconds')
print('Problem solved in ', solver.iterations(), ' iterations')


----------- 고정 재료 목록 -----------
['돼지고기, 등심, 생것', 142, 71.1, 23.33, 4.58, 1.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4, 0.43, 25, 222, 370, 42, 1.56, 0.02, 0.002, 23.09, 0.99, 0.2, 3, '3', '0', 0.627, 0.147, 3.168, 7.323, 0.0, 3.168, 0.598, 0.0, 0.046, 1.78, 1, 1, 0.0, 0.37, '0.75', 0.25, 0.0, 0.25, 0.27, 0.27, 0.0, 0.03, 0.0, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 20838, 10559, 976.0, 1787.0, 1739.0, 568, 921.0, 1061.0, 249, 1172.0, 871, 1214.0, 786.0, 279, 1359.0, 2019.0, 3010.0, 1165.0, 784.0, 877.0, 0.0, 54.39, 4.32, 0.66, 1.68, 0.0, 0.0, 0.0, 4.67, 8.23, 0.0, 76.18, 2.23, 987.75, 21.78, 512.18, 7.46, 2.14, 0.0, 59.08, 0.0, 2.62, 1.91, 1.19, 113.83, 0.0, 1597.66, 155.19, 36.67, 0.0, 0.0, 0.71, 634.32, 27.58, 0.0, 26.07, 5.17, 10.46, 0.0, 0.0, 0.0, 6.6, 0.0, 0.04, 0.67, 0.03, 15.45, 6.83, 4.32, 0.1, 0.0]
['달걀, 생것', 136, 77.1, 12.91, 8.25, 0.95, 0.79, 0.11, 0.0, 0.11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 47, 1.92, 12, 173, 127, 139, 1.2, 0.042, 0.031, 32.91, 4.74, 29.87, 

In [40]:
result_list = [0 for _ in range(132)]
for i in range(len(input_list)):
    temp = [float(x) * 0.01 * float(calculated_recipe[i][1]) for x in input_list[i][1:]]
    result_list = result_list + temp[1:]


In [42]:
calculated_recipe

[('돼지고기, 등심, 생것', 142),
 ('달걀, 생것', 5278),
 ('멥쌀, 백미, 밥', 146),
 ('고구마, 생것', 147),
 ('호박, 단호박, 생것', 57)]

In [41]:
result_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 100.96199999999999,
 33.1286,
 6.5036000000000005,
 1.562,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.68,
 0.6106,
 35.5,
 315.24,
 525.4,
 59.64,
 2.2152000000000003,
 0.0284,
 0.00284,
 32.7878,
 1.4058000000000002,
 0.28400000000000003,
 4.26,
 4.26,
 0.0,
 0.89034,
 0.20873999999999998,
 4.49856,
 10.398660000000001,
 0.0,
 4.49856,
 0.84916,
 0.0,
 0.06532,
 2.5276,
 1.42,
 1.42,
 0.0,
 0.5254,
 1.065,
 0.355,
 0.0,
 0.355,
 0.3834,
 0.